In [ ]:
import tensorflow as tf
# tf.test.is_gpu_available()
tf.config.list_physical_devices('GPU')


In [ ]:
import tensorflow as tf
# my custom lib
from frankModel import LeNet
from frankModel import AlexNet
from frankModel import VGG16
from frankModel import InceptionV1
from frankModel import ResNet50
from frankModel import EfficientNetV2_S


from small_dataset import MNIST
from small_dataset import CIFAR10
from small_dataset import CIFAR100

In [ ]:
dataset = CIFAR10(info=True).addChannel().tocategorical().Done()

In [ ]:
import tensorflow as tf

#取得模型架構
MyNet = EfficientNetV2_S(datasetName=dataset.className,input_shape=(None,None,3) ,classes=len(dataset.train_y[0]))


In [ ]:
initial_learning_rate = 1e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)

#訓練
# compile  #在每層 layer 和 compile 都可自動尋找超參數
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

with strategy.scope():
    #取得模型架構
    MyNet = EfficientNetV2_S(datasetName=dataset.className,input_shape=(32,32,3) ,classes=len(dataset.train_y[0]))
    
    MyNet.model.compile(
        #learning_rate=0.01
        optimizer= tf.keras.optimizers.Adam(learning_rate=lr_schedule,epsilon=1e-09),
        loss= 'categorical_crossentropy',
        metrics=['accuracy']
    )

# fit
history  = \
MyNet.model.fit(
    x = dataset.train_x,
    y = dataset.train_y,
    epochs = 30,
    batch_size = 64,
    validation_data = (dataset.test_x ,dataset.test_y)
)


In [ ]:
#取得訓練結果
#   save the entire model as a single file
#   model = keras.models.load_model("path_to_my_model")
# model.save(f"{save_model_dir}")
MyNet.outputHelper.saveModel()

In [ ]:
history.history

In [ ]:
# print(history.history)
r = history.history
MyNet.outputHelper.drawTrainProcess(r)